# DataFrames

A simple way to think of a `DataFrame` in Spark is to see it as a distributed table of data or a distributed CSV file. However, this distributed table of data or CSV file comes highly glorified with bells and whistles and battery included. The highly prized features of Spark DataFrames is that they are not unlike `R` or `pandas` data frames and you can issue SQL-like commands against them. 

As a side note, there are three main distributed data structures in Spark. 

* `RDD`
* `DataFrame`
* `DataSet`

The RDD data structure is the original distributed data structure and the records can be anything. RDDs were very friendly to experienced data engineers and programmers. DataFrames are a movement away RDDs, and provide tabular structure to records and made Spark accessible to other types of data programmers (such as those who are comfortable with SQL). Still, DataFrames were too generic, and DataSets were created to have the tabular structure of DataFrames where the records could be specifically defined. A DataFrame is just a DataSet, where the records are of the type `Record`, and a DataSet is said to be a strongly-typed, user-defined distributed, tabular data structure.

It is said, when you are using a RDD, you are describing `how` you are doing something and when you are using a DataFrame or DataSet, you are describing `what` you are doing. The `how` you are doing something relates to the `imperative` programming paradigm, and the `what` you are doing relates to the `functional` programming paradigm. It is argued that code that tells `what` you are doing is easier to understand than code that tells `how` you are doing it. However, which one of these approaches is easier to understand differs from person to person. Sometimes, functional programming style of coding results in highly nested code. Take the example below assuming that we are composing behavior through the composition of functions.

```python
is(this(how(we(want(to(code))))))
```

## Acquiring a DataFrame

### Convert Pandas DataFrame to Spark DataFrame

The easiest way to get a Spark DataFrame is to convert a pandas DataFrame to a Spark one. There's a convenience method from the `sqlContext` to do so, `createDataFrame()`. Below, the pandas DataFrame is set to `pdf` and the Spark DataFrame is set to `sdf`.

In [1]:
import pandas as pd
from random import randint

n_cols = 10
n_rows = 10

pdf = pd.DataFrame([tuple([c for c in range(n_cols)]) for r in range(n_rows)], columns=[f'x{i}' for i in range(n_cols)])
sdf = sqlContext.createDataFrame(pdf)

In [2]:
pdf

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0,1,2,3,4,5,6,7,8,9
1,0,1,2,3,4,5,6,7,8,9
2,0,1,2,3,4,5,6,7,8,9
3,0,1,2,3,4,5,6,7,8,9
4,0,1,2,3,4,5,6,7,8,9
5,0,1,2,3,4,5,6,7,8,9
6,0,1,2,3,4,5,6,7,8,9
7,0,1,2,3,4,5,6,7,8,9
8,0,1,2,3,4,5,6,7,8,9
9,0,1,2,3,4,5,6,7,8,9


Notice how the Spark DataFrame's records are of type `Row`? The `Row` behaves just like Python's `tuple` and `dictionary` types.

In [3]:
sdf.collect()

[Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9)]

Let's grab the first row of this DataFrame and see how we can interact with it.

In [4]:
row = sdf.take(1)[0]
row

Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9)

We can access the fields of a `Row` via index notation (like a tuple).

In [5]:
for i in range(len(row)):
    print(row[i])

0
1
2
3
4
5
6
7
8
9


We can also reference the values by keys in a row (like a dictionary).

In [6]:
for i in range(len(row)):
    key = f'x{i}'
    print(row[key])

0
1
2
3
4
5
6
7
8
9


Sometimes, it's just better to convert the row to a dictionary and then use our Python knowledge of iterating and manipulating dictionaries.

In [7]:
for k, v in row.asDict().items():
    print(f'{k}: {v}')

x0: 0
x1: 1
x2: 2
x3: 3
x4: 4
x5: 5
x6: 6
x7: 7
x8: 8
x9: 9


Enough of `Rows`, let's get back to the Spark DataFrame. We can display the contents of a Spark DataFrame with `show()`.

In [8]:
sdf.show()

+---+---+---+---+---+---+---+---+---+---+
| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+---+---+---+---+---+---+---+---+---+---+
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
+---+---+---+---+---+---+---+---+---+---+



What if we want to inspect the schema of a DataFrame? 

In [9]:
sdf.printSchema()

root
 |-- x0: long (nullable = true)
 |-- x1: long (nullable = true)
 |-- x2: long (nullable = true)
 |-- x3: long (nullable = true)
 |-- x4: long (nullable = true)
 |-- x5: long (nullable = true)
 |-- x6: long (nullable = true)
 |-- x7: long (nullable = true)
 |-- x8: long (nullable = true)
 |-- x9: long (nullable = true)



### Convert a RDD to DataFrame

A RDD can be converted to a DataFrame, however, we need to create a schema. Below is a concise way of creating a schema for a RDD. Notice that `createDataFrame()` is overloaded? Before, we passed in a pandas DataFrame. Here, we pass in a RDD and a schema.

In [10]:
from random import randint
from pyspark.sql.types import *

n_cols = 10
n_rows = 10

rdd = sc.parallelize([[c for c in range(n_cols)] for r in range(n_rows)])

schema = StructType([StructField(f'x{i}', IntegerType(), True) for i in range(n_cols)])
df = sqlContext.createDataFrame(rdd, schema)

If the above example is too concise, let's do build the schema manually. The schema is defined by a `StructType` and the `StructType` is based off of a list of `StructFields`.

In [11]:
struct_fields = [
    StructField('x0', IntegerType(), True),
    StructField('x1', IntegerType(), True),
    StructField('x2', IntegerType(), True),
    StructField('x3', IntegerType(), True),
    StructField('x4', IntegerType(), True),
    StructField('x5', IntegerType(), True),
    StructField('x6', IntegerType(), True),
    StructField('x7', IntegerType(), True),
    StructField('x8', IntegerType(), True),
    StructField('x9', IntegerType(), True)
]

struct_type = StructType(struct_fields)
df = sqlContext.createDataFrame(rdd, struct_type)

Let's inspect the RDD. It's a list of lists (of integers).

In [12]:
rdd.collect()

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

Let's inspect the DataFrame. It's a list of `Rows`.

In [13]:
df.collect()

[Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9),
 Row(x0=0, x1=1, x2=2, x3=3, x4=4, x5=5, x6=6, x7=7, x8=8, x9=9)]

Here's a display of the DataFrame.

In [14]:
df.show()

+---+---+---+---+---+---+---+---+---+---+
| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+---+---+---+---+---+---+---+---+---+---+
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|
+---+---+---+---+---+---+---+---+---+---+



And let's inspect the schema.

In [15]:
df.printSchema()

root
 |-- x0: integer (nullable = true)
 |-- x1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- x3: integer (nullable = true)
 |-- x4: integer (nullable = true)
 |-- x5: integer (nullable = true)
 |-- x6: integer (nullable = true)
 |-- x7: integer (nullable = true)
 |-- x8: integer (nullable = true)
 |-- x9: integer (nullable = true)



### Convert JSON data to Spark DataFrame

We have seen how to create a Spark DataFrame from a pandas DataFrame or a RDD. Let's see how we can create a Spark DataFrame from reading a `JSON` file. First, let's upload the JSON file to `HDFS`.

In [16]:
%%sh
hdfs dfs -copyFromLocal -f /root/ipynb/people.json /people.json

2020-11-07 04:50:32,012 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


Use the `sqlContext.read.json()` method to read the JSON file from HDFS.

In [17]:
df = sqlContext.read.json('hdfs://localhost/people.json')

Let's see what's inside the Spark DataFrame. Notice how the Spark DataFrame is still tabular or table-ish? We know that JSON is a highly nested structure, and so where there's nesting, only the top-level keys are mapped to the fields/columns of the DataFrame.

In [18]:
df.show()

+--------------------+---+----------+------+---+---------+-----+--------------------+------+
|             address|age|first_name|height| id|last_name| male|              sports|weight|
+--------------------+---+----------+------+---+---------+-----+--------------------+------+
|[Washington, DC, ...| 27|      John|   6.5|  1|      Doe| true|    [hockey, tennis]| 155.5|
|[Washington, DC, ...| 22|      Jane|   5.7|  2|    Smith|false|[basketball, tennis]| 135.5|
|[Los Angeles, CA,...| 25|      Jack|   6.6|  3|    Smith| true|  [baseball, soccer]| 175.5|
|[Los Angeles, CA,...| 18|     Janet|   5.5|  4|      Doe|false|    [judo, baseball]| 125.5|
+--------------------+---+----------+------+---+---------+-----+--------------------+------+



But then, inspect the schema. The schema reflects the nested JSON data. 

In [19]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- zip: long (nullable = true)
 |-- age: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- height: double (nullable = true)
 |-- id: long (nullable = true)
 |-- last_name: string (nullable = true)
 |-- male: boolean (nullable = true)
 |-- sports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- weight: double (nullable = true)



## DataFrame operations

What can we actually do with a Spark DataFrame? Can we do amazing things with DataFrames as with RDDs?

### Create data

Let's create a dummy Spark DataFrame first.

In [20]:
import pandas as pd
from random import randint, choice

def generate_num(col):
    if col == 3:
        p = randint(1, 100)
        if p < 70:
            return None
        return randint(1, 10)
    return randint(1, 10)

def generate_height():
    return choice(['tall', 'short'])

n_cols = 10
n_rows = 10

pdf = pd.DataFrame(
    [tuple([generate_height()] + [generate_num(c) for c in range(n_cols)]) for r in range(n_rows)], 
    columns=['height'] + [f'x{i}' for i in range(n_cols)])
sdf = sqlContext.createDataFrame(pdf)

In [21]:
pdf

,height,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,short,1,2,4,NaN,5,3,2,3,8,2
1,tall,6,6,4,2.0,8,9,4,1,7,1
2,tall,2,2,4,NaN,10,2,8,8,6,5
3,short,8,5,3,NaN,2,5,6,5,6,6
4,tall,2,5,5,NaN,3,10,10,1,8,4
5,tall,7,2,9,NaN,10,1,10,4,4,1
6,tall,4,10,9,8.0,9,1,10,2,1,10
7,short,2,6,3,9.0,1,9,10,1,4,6
8,short,6,4,5,NaN,5,8,7,8,7,5
9,tall,4,1,10,8.0,8,9,7,10,6,6


### Select

We can use `select()` to grab specific columns from a Spark DataFrame.

In [22]:
sdf.select('x0').show()

+---+
| x0|
+---+
|  1|
|  6|
|  2|
|  8|
|  2|
|  7|
|  4|
|  2|
|  6|
|  4|
+---+



What if we want to grab multiple columns?

In [23]:
sdf.select('x0', 'x1').show()

+---+---+
| x0| x1|
+---+---+
|  1|  2|
|  6|  6|
|  2|  2|
|  8|  5|
|  2|  5|
|  7|  2|
|  4| 10|
|  2|  6|
|  6|  4|
|  4|  1|
+---+---+



We can also select specific columns as follows using a `Column` object. You will see both variants in the wild; one with a list of column names, and the one below referencing the `Column` itself (from the Data Frame). When would you want to use the literal column name versus the object `Column`? Look below. When we need to transform the values in the column inline, we have to use the `Column` object.

In [24]:
sdf.select(sdf['x0'], sdf['x1']).show()

+---+---+
| x0| x1|
+---+---+
|  1|  2|
|  6|  6|
|  2|  2|
|  8|  5|
|  2|  5|
|  7|  2|
|  4| 10|
|  2|  6|
|  6|  4|
|  4|  1|
+---+---+



We can even modify values that we are retrieving. Below, we multiply the first column we want by two and the second column we want by three. Observe the column names. Yuck!

In [25]:
sdf.select(sdf['x0'] * 2, sdf['x1'] * 3).show()

+--------+--------+
|(x0 * 2)|(x1 * 3)|
+--------+--------+
|       2|       6|
|      12|      18|
|       4|       6|
|      16|      15|
|       4|      15|
|      14|       6|
|       8|      30|
|       4|      18|
|      12|      12|
|       8|       3|
+--------+--------+



We can fix the column names with `alias()`. Uh-oh! Notice how the parentheses are creeping in? Is this style of coding for `what` we are doing clear in intention?

In [26]:
sdf.select((sdf['x0'] * 2).alias('y0'), (sdf['x1'] * 3).alias('y1')).show()

+---+---+
| y0| y1|
+---+---+
|  2|  6|
| 12| 18|
|  4|  6|
| 16| 15|
|  4| 15|
| 14|  6|
|  8| 30|
|  4| 18|
| 12| 12|
|  8|  3|
+---+---+



I suppose a little formatting might help.

In [27]:
sdf.select(
    (sdf['x0'] * 2).alias('y0'), 
    (sdf['x1'] * 3).alias('y1'))\
    .show()

+---+---+
| y0| y1|
+---+---+
|  2|  6|
| 12| 18|
|  4|  6|
| 16| 15|
|  4| 15|
| 14|  6|
|  8| 30|
|  4| 18|
| 12| 12|
|  8|  3|
+---+---+



We can also apply boolean expressions with `select()`.

In [28]:
sdf.select(sdf['x0'] > 5).show()

+--------+
|(x0 > 5)|
+--------+
|   false|
|    true|
|   false|
|    true|
|   false|
|    true|
|   false|
|   false|
|    true|
|   false|
+--------+



How do we get distinct values?

In [29]:
sdf.select('x0').distinct().show()

+---+
| x0|
+---+
|  7|
|  6|
|  1|
|  8|
|  2|
|  4|
+---+



We can do a set difference operation with `subtract()` as follows. We use `distinct()` to enforce uniqueness.

In [30]:
x0 = sdf.select('x0')
x1 = sdf.select('x1')
diff = x0.subtract(x1)
diff.distinct().show()

+---+
| x0|
+---+
|  7|
|  8|
+---+



We can also add columns.

In [31]:
sdf.withColumn('height_truthy', sdf['height'] == 'tall').show()

+------+---+---+---+---+---+---+---+---+---+---+-------------+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|height_truthy|
+------+---+---+---+---+---+---+---+---+---+---+-------------+
| short|  1|  2|  4|NaN|  5|  3|  2|  3|  8|  2|        false|
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|         true|
|  tall|  2|  2|  4|NaN| 10|  2|  8|  8|  6|  5|         true|
| short|  8|  5|  3|NaN|  2|  5|  6|  5|  6|  6|        false|
|  tall|  2|  5|  5|NaN|  3| 10| 10|  1|  8|  4|         true|
|  tall|  7|  2|  9|NaN| 10|  1| 10|  4|  4|  1|         true|
|  tall|  4| 10|  9|8.0|  9|  1| 10|  2|  1| 10|         true|
| short|  2|  6|  3|9.0|  1|  9| 10|  1|  4|  6|        false|
| short|  6|  4|  5|NaN|  5|  8|  7|  8|  7|  5|        false|
|  tall|  4|  1| 10|8.0|  8|  9|  7| 10|  6|  6|         true|
+------+---+---+---+---+---+---+---+---+---+---+-------------+



Dropping columns is achieved with `drop()`.

In [32]:
sdf.withColumn('height_truthy', sdf['height'] == 'tall').columns

['height',
 'x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'height_truthy']

In [33]:
sdf.withColumn('height_truthy', sdf['height'] == 'tall').drop('height_truthy').columns

['height', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']

### Filtering

How do we filter records in a Spark DataFrame? 

In [34]:
sdf.filter(sdf['x0'] > 5).show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
| short|  8|  5|  3|NaN|  2|  5|  6|  5|  6|  6|
|  tall|  7|  2|  9|NaN| 10|  1| 10|  4|  4|  1|
| short|  6|  4|  5|NaN|  5|  8|  7|  8|  7|  5|
+------+---+---+---+---+---+---+---+---+---+---+



Multiple filters?

In [35]:
sdf.filter(sdf['x0'] > 5).filter(sdf['x1'] > 5).show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
+------+---+---+---+---+---+---+---+---+---+---+



What about dropping duplicates?

In [36]:
sdf.select('x0').dropDuplicates().show()

+---+
| x0|
+---+
|  7|
|  6|
|  1|
|  8|
|  2|
|  4|
+---+



### Ordering

How do we order the records?

In [37]:
sdf.orderBy('x0').show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
| short|  1|  2|  4|NaN|  5|  3|  2|  3|  8|  2|
| short|  2|  6|  3|9.0|  1|  9| 10|  1|  4|  6|
|  tall|  2|  5|  5|NaN|  3| 10| 10|  1|  8|  4|
|  tall|  2|  2|  4|NaN| 10|  2|  8|  8|  6|  5|
|  tall|  4| 10|  9|8.0|  9|  1| 10|  2|  1| 10|
|  tall|  4|  1| 10|8.0|  8|  9|  7| 10|  6|  6|
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
| short|  6|  4|  5|NaN|  5|  8|  7|  8|  7|  5|
|  tall|  7|  2|  9|NaN| 10|  1| 10|  4|  4|  1|
| short|  8|  5|  3|NaN|  2|  5|  6|  5|  6|  6|
+------+---+---+---+---+---+---+---+---+---+---+



And ordering by multiple columns?

In [38]:
sdf.orderBy('height', 'x0').show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
| short|  1|  2|  4|NaN|  5|  3|  2|  3|  8|  2|
| short|  2|  6|  3|9.0|  1|  9| 10|  1|  4|  6|
| short|  6|  4|  5|NaN|  5|  8|  7|  8|  7|  5|
| short|  8|  5|  3|NaN|  2|  5|  6|  5|  6|  6|
|  tall|  2|  5|  5|NaN|  3| 10| 10|  1|  8|  4|
|  tall|  2|  2|  4|NaN| 10|  2|  8|  8|  6|  5|
|  tall|  4|  1| 10|8.0|  8|  9|  7| 10|  6|  6|
|  tall|  4| 10|  9|8.0|  9|  1| 10|  2|  1| 10|
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
|  tall|  7|  2|  9|NaN| 10|  1| 10|  4|  4|  1|
+------+---+---+---+---+---+---+---+---+---+---+



And ordering descendingly?

In [39]:
sdf.orderBy(sdf['height'].desc(), sdf['x0'].desc()).show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
|  tall|  7|  2|  9|NaN| 10|  1| 10|  4|  4|  1|
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
|  tall|  4|  1| 10|8.0|  8|  9|  7| 10|  6|  6|
|  tall|  4| 10|  9|8.0|  9|  1| 10|  2|  1| 10|
|  tall|  2|  5|  5|NaN|  3| 10| 10|  1|  8|  4|
|  tall|  2|  2|  4|NaN| 10|  2|  8|  8|  6|  5|
| short|  8|  5|  3|NaN|  2|  5|  6|  5|  6|  6|
| short|  6|  4|  5|NaN|  5|  8|  7|  8|  7|  5|
| short|  2|  6|  3|9.0|  1|  9| 10|  1|  4|  6|
| short|  1|  2|  4|NaN|  5|  3|  2|  3|  8|  2|
+------+---+---+---+---+---+---+---+---+---+---+



### Missing values

How do we handle missing values? First, we can drop them.

In [40]:
sdf.dropna().show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
|  tall|  6|  6|  4|2.0|  8|  9|  4|  1|  7|  1|
|  tall|  4| 10|  9|8.0|  9|  1| 10|  2|  1| 10|
| short|  2|  6|  3|9.0|  1|  9| 10|  1|  4|  6|
|  tall|  4|  1| 10|8.0|  8|  9|  7| 10|  6|  6|
+------+---+---+---+---+---+---+---+---+---+---+



Or maybe we want to set missing values to zero?

In [41]:
sdf.fillna(-1).show()

+------+---+---+---+----+---+---+---+---+---+---+
|height| x0| x1| x2|  x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+----+---+---+---+---+---+---+
| short|  1|  2|  4|-1.0|  5|  3|  2|  3|  8|  2|
|  tall|  6|  6|  4| 2.0|  8|  9|  4|  1|  7|  1|
|  tall|  2|  2|  4|-1.0| 10|  2|  8|  8|  6|  5|
| short|  8|  5|  3|-1.0|  2|  5|  6|  5|  6|  6|
|  tall|  2|  5|  5|-1.0|  3| 10| 10|  1|  8|  4|
|  tall|  7|  2|  9|-1.0| 10|  1| 10|  4|  4|  1|
|  tall|  4| 10|  9| 8.0|  9|  1| 10|  2|  1| 10|
| short|  2|  6|  3| 9.0|  1|  9| 10|  1|  4|  6|
| short|  6|  4|  5|-1.0|  5|  8|  7|  8|  7|  5|
|  tall|  4|  1| 10| 8.0|  8|  9|  7| 10|  6|  6|
+------+---+---+---+----+---+---+---+---+---+---+



### Group by

How do we do grouping?

In [42]:
sdf.groupBy('height').count().show()

+------+-----+
|height|count|
+------+-----+
|  tall|    6|
| short|    4|
+------+-----+



We can also do aggregations `agg()` after a group-by.

In [43]:
sdf.groupBy('height').agg({'x0': 'mean'}).show()

+------+-----------------+
|height|          avg(x0)|
+------+-----------------+
|  tall|4.166666666666667|
| short|             4.25|
+------+-----------------+



Multiple aggregations over different columns.

In [44]:
sdf.groupBy('height').agg({'x0': 'mean', 'x1': 'mean'}).show()

+------+-----------------+-----------------+
|height|          avg(x0)|          avg(x1)|
+------+-----------------+-----------------+
|  tall|4.166666666666667|4.333333333333333|
| short|             4.25|             4.25|
+------+-----------------+-----------------+



Oh-uh, it seems if we want multiple aggregation over the same column, the results will not compute.

In [45]:
sdf.groupBy('height').agg({'x0': 'mean', 'x0': 'stddev'}).show()

+------+-----------------+
|height|       stddev(x0)|
+------+-----------------+
|  tall|2.041241452319315|
| short|3.304037933599835|
+------+-----------------+



Here are some group functions.

In [46]:
from pyspark.sql.functions import countDistinct, avg, stddev

sdf.select(avg('x0'), stddev('x0'), countDistinct('x0')).show()

+-------+-----------------+------------------+
|avg(x0)|  stddev_samp(x0)|count(DISTINCT x0)|
+-------+-----------------+------------------+
|    4.2|2.440400695696417|                 6|
+-------+-----------------+------------------+



### Cross-tabulation

If we wanted to do cross-tabulation, we need to use `crosstab()`.

In [47]:
sdf.crosstab('height', 'x1').show()

+---------+---+---+---+---+---+---+
|height_x1|  1| 10|  2|  4|  5|  6|
+---------+---+---+---+---+---+---+
|    short|  0|  0|  1|  1|  1|  1|
|     tall|  1|  1|  2|  0|  1|  1|
+---------+---+---+---+---+---+---+



### Statistics

I want statistics. Ugh, the standard deviation has too much precision. How can we fix the precision?

In [48]:
sdf.describe('x0', 'x1').show()

+-------+-----------------+------------------+
|summary|               x0|                x1|
+-------+-----------------+------------------+
|  count|               10|                10|
|   mean|              4.2|               4.3|
| stddev|2.440400695696417|2.7100635498903793|
|    min|                1|                 1|
|    max|                8|                10|
+-------+-----------------+------------------+



With some coding gymnastics, we need to cast the columns `x0` and `x1` to `DoubleType` and then use `format_number()` to specify the precision.

In [49]:
from pyspark.sql.functions import format_number, col

sdf.describe('x0', 'x1')\
    .withColumn('x0', col('x0').cast(DoubleType()))\
    .withColumn('x1', col('x1').cast(DoubleType()))\
    .select('summary', format_number('x0', 2).alias('x0'), format_number('x1', 2).alias('x1'))\
    .show()

+-------+-----+-----+
|summary|   x0|   x1|
+-------+-----+-----+
|  count|10.00|10.00|
|   mean| 4.20| 4.30|
| stddev| 2.44| 2.71|
|    min| 1.00| 1.00|
|    max| 8.00|10.00|
+-------+-----+-----+



Here's another way using `selectExpr()`.

In [50]:
sdf.describe('x0', 'x1')\
    .selectExpr('summary', 'cast(x0 as double) as x0', 'cast(x1 as double) as x1')\
    .select(
        'summary', 
        format_number('x0', 2).alias('x0'), 
        format_number('x1', 2).alias('x1'))\
    .show()

+-------+-----+-----+
|summary|   x0|   x1|
+-------+-----+-----+
|  count|10.00|10.00|
|   mean| 4.20| 4.30|
| stddev| 2.44| 2.71|
|    min| 1.00| 1.00|
|    max| 8.00|10.00|
+-------+-----+-----+



### Sampling

We can also sample.

In [51]:
sdf.sample(True, 0.5, 37).show()

+------+---+---+---+---+---+---+---+---+---+---+
|height| x0| x1| x2| x3| x4| x5| x6| x7| x8| x9|
+------+---+---+---+---+---+---+---+---+---+---+
|  tall|  2|  2|  4|NaN| 10|  2|  8|  8|  6|  5|
|  tall|  2|  5|  5|NaN|  3| 10| 10|  1|  8|  4|
+------+---+---+---+---+---+---+---+---+---+---+



### User defined function (UDF)

If you have complicated logic to transform a column, you can use `User-Defined Functions` or `UDFs`. To create a UDF, define the function that will do the transformation first. Below, we define `times_two()` to take in a number input and return that number times two. The second thing you need to do is create the UDF using `udf()`, which requires two arguments:

* the function that will do the transform 
* the return type

The return type comes from the `pyspark.sql.types` module. Finally, you can apply your UDF as an argument to `select()`.

In [52]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def times_two(num):
    return num * 2

times_two_udf = udf(times_two, IntegerType())
sdf.select('x0', times_two_udf('x0').alias('times_two')).show()

+---+---------+
| x0|times_two|
+---+---------+
|  1|        2|
|  6|       12|
|  2|        4|
|  8|       16|
|  2|        4|
|  7|       14|
|  4|        8|
|  2|        4|
|  6|       12|
|  4|        8|
+---+---------+



A UDF can also accept multiple arguments. Here's an example.

In [53]:
def add_them(a, b):
    return a + b

add_them_udf = udf(add_them, IntegerType())
sdf.select('x0', 'x1', add_them_udf('x0', 'x1').alias('add_them')).show()

+---+---+--------+
| x0| x1|add_them|
+---+---+--------+
|  1|  2|       3|
|  6|  6|      12|
|  2|  2|       4|
|  8|  5|      13|
|  2|  5|       7|
|  7|  2|       9|
|  4| 10|      14|
|  2|  6|       8|
|  6|  4|      10|
|  4|  1|       5|
+---+---+--------+



### User defined function (UDF) with annotation

It's probably easiest to use the `@udf` decorator on a function. Note that the `@udf` decorator is parameterized; we have to specify the return type.

In [54]:
from pyspark.sql.functions import udf

@udf('int')
def times_three(num):
    return num * 3

sdf.select('x0', times_three('x0').alias('times_three')).show()

+---+-----------+
| x0|times_three|
+---+-----------+
|  1|          3|
|  6|         18|
|  2|          6|
|  8|         24|
|  2|          6|
|  7|         21|
|  4|         12|
|  2|          6|
|  6|         18|
|  4|         12|
+---+-----------+

